# Senior Design Team 1725, Classifying AIS, Alternative Model Training
Yolov8 Testing- Ziek

This file is a juptyer notebook containing the training results using Yolov8 as a alternative solution model while Joey and Landon work with Yolov11

## Dataset Mounting (Sharepoint):
Before getting to the model we first need to access the dataset. This is done using our sharepoint/OneDrive to colab.

Currently rlcone config must be first run *locally* then once obtaining the API key it can be run on colab. This is so that the mount works properly, which requires web browser access (this is not possible in colab)

In [ ]:
!wget https://downloads.rclone.org/v1.69.0/rclone-v1.69.0-linux-amd64.deb
!apt install ./rclone-v1.69.0-linux-amd64.deb

--2025-02-19 21:32:40--  https://downloads.rclone.org/v1.69.0/rclone-v1.69.0-linux-amd64.deb
Resolving downloads.rclone.org (downloads.rclone.org)... 95.217.6.16, 2a01:4f9:c012:7154::1
Connecting to downloads.rclone.org (downloads.rclone.org)|95.217.6.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23665802 (23M) [application/vnd.debian.binary-package]
Saving to: ‘rclone-v1.69.0-linux-amd64.deb’

rclone-v1.69.0-linu 100%[===================>]  22.57M  8.34MB/s    in 2.7s    

2025-02-19 21:32:43 (8.34 MB/s) - ‘rclone-v1.69.0-linux-amd64.deb’ saved [23665802/23665802]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'rclone' instead of './rclone-v1.69.0-linux-amd64.deb'
The following NEW packages will be installed:
  rclone
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 0 B/23.7 MB of archives.
After this operation, 71.0 MB of additional disk space will be used.
Get

In [ ]:
# We will need to pass some arguments for this to work,
# so make sure we still have the api key
# (Note: you need to copy the ENTIRE struct, as the values are read as an array)

# Also, make sure to clear the output! We don't want to have
# loose api keys floating around the internet!
!rclone config

#current API key (update this regularly): {"access_token":"eyJ0eXAiOiJKV1QiLCJub25jZSI6IjJUMEU3bExWMENfYlM3ZnBuR3JHeEZQQXBaZ0RIS2RuNEU3Y1M2TThfYmMiLCJhbGciOiJSUzI1NiIsIng1dCI6ImltaTBZMnowZFlLeEJ0dEFxS19UdDVoWUJUayIsImtpZCI6ImltaTBZMnowZFlLeEJ0dEFxS19UdDVoWUJUayJ9.eyJhdWQiOiIwMDAwMDAwMy0wMDAwLTAwMDAtYzAwMC0wMDAwMDAwMDAwMDAiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC83Y2Y0OGQ0NS0zZGRiLTQzODktYTljMS1jMTE1NTI2ZWI1MmUvIiwiaWF0IjoxNzM5OTk4ODAxLCJuYmYiOjE3Mzk5OTg4MDEsImV4cCI6MTc0MDAwMzExNSwiYWNjdCI6MCwiYWNyIjoiMSIsImFpbyI6IkFZUUFlLzhaQUFBQWxWeXdsMnpBTU1tMmRVbXA0TkJIeG4zcE9FTzE4MU1TVVk4MnRKNlYvdFlnamxiUEdDeDBrbUVPaXBlVHZObzFKNXRad2QxYTYyb0FCNTZJeTRwejl5bkJGbHd2eXZ3d292WHA1WTArMkNyMHFwUW02Qjd4TkwxenNwTnBJRXZsN0RRUXJ0OHRJQTU3ZUQrVnFqL3IyNTBRa0UvVXhqajZuR3VMN2dVRDVERT0iLCJhbXIiOlsicHdkIiwibWZhIl0sImFwcF9kaXNwbGF5bmFtZSI6InJjbG9uZSIsImFwcGlkIjoiYjE1NjY1ZDktZWRhNi00MDkyLTg1MzktMGVlYzM3NmFmZDU5IiwiYXBwaWRhY3IiOiIxIiwiZmFtaWx5X25hbWUiOiJNb3Jlbm8iLCJnaXZlbl9uYW1lIjoiRXplcXVpZWwiLCJpZHR5cCI6InVzZXIiLCJpcGFkZHIiOiIxODQuNTYuMTI0LjI1IiwibmFtZSI6Ik1vcmVubywgRXplcXVpZWwgQWxleGFuZGVyIiwib2lkIjoiZTc1ZGIyZGQtM2U0NC00NjFlLWI3YjUtMjcwNjhhZTM1YWNmIiwib25wcmVtX3NpZCI6IlMtMS01LTIxLTI0OTU1OTY0NDItMTYxMTYzNTc1MC0yNjk0NTc5MTU1LTIyNDA5NzQiLCJwbGF0ZiI6IjMiLCJwdWlkIjoiMTAwMzIwMDEzMzdGNzc1OCIsInJoIjoiMS5BUklBUlkzMGZOczlpVU9wd2NFVlVtNjFMZ01BQUFBQUFBQUF3QUFBQUFBQUFBQVNBSlFTQUEuIiwic2NwIjoiRmlsZXMuUmVhZCBGaWxlcy5SZWFkLkFsbCBGaWxlcy5SZWFkV3JpdGUgRmlsZXMuUmVhZFdyaXRlLkFsbCBTaXRlcy5SZWFkLkFsbCBVc2VyLlJlYWQgcHJvZmlsZSBvcGVuaWQgZW1haWwiLCJzaWQiOiIwMDIxNzI5OS04ZDdlLTA5NzctNzEzMS0xYjkzNDMzNzJjOTciLCJzdWIiOiJmQmtEUHpndE94SXRZNnEzZ3VRbGZSWlNZb3RpYWhGMUtOanFpQ2VodC04IiwidGVuYW50X3JlZ2lvbl9zY29wZSI6Ik5BIiwidGlkIjoiN2NmNDhkNDUtM2RkYi00Mzg5LWE5YzEtYzExNTUyNmViNTJlIiwidW5pcXVlX25hbWUiOiJlYW02MjkyQHBzdS5lZHUiLCJ1cG4iOiJlYW02MjkyQHBzdS5lZHUiLCJ1dGkiOiJmQzdWLTIzaVBrcWlqNThhbDBoWUFBIiwidmVyIjoiMS4wIiwid2lkcyI6WyJiNzlmYmY0ZC0zZWY5LTQ2ODktODE0My03NmIxOTRlODU1MDkiXSwieG1zX2Z0ZCI6IkVsaUdUVVVtRHNCdjl2RWN5TFY3SzhLZVFldUplTnN4cy1UYWFpR010ZW8iLCJ4bXNfaWRyZWwiOiIxIDI4IiwieG1zX3N0Ijp7InN1YiI6IjJ6NmEyVEhQUFM2dzB1Z0xCU0FDeGJKX1pLNk8xZkU0NU1MMDBYdEF3SDAifSwieG1zX3RjZHQiOjE0MTkwMTY5MjF9.jpQEUVE21S6Ar1bda0R3L3_RcwxbnTauX1ejAliBVFG_yt5mP3QejZ4JRNCkbDzrI4YXaBzTYt3_jnlt8qgNI1bWefOTPGcDXjRsy-PjZQOadFj5o-fSiIjlILmpznerqRoLlM48NS93iIOmTt01w8yq_PaVXzfszpVQLOKxY_b4LuOVjyoGaSXn81iRpGTB1hH3ylIDvn-195HNRHTncKOJEhFKxUna6FLTleQjE8SBrsGbypqg89zY6gs4oWgMAIdMO_4mczIIc63g0gvhVsuGBL40kaLh1GsAIVJ6s_1i5qu2L8dR1LrImXmDJDcsl8RTDNhrAep8jNfxUE7HCw","token_type":"Bearer","refresh_token":"1.ARIARY30fNs9iUOpwcEVUm61LtllVrGm7ZJAhTkO7Ddq_VkSAJQSAA.AgABAwEAAABVrSpeuWamRam2jAF1XRQEAwDs_wUA9P_MUfkqCcDkchEwJPK7pe1xDNF3YxOwWxPwQ1hNij29qlV35pudnjjIgC8rsfwcrMilQTvQ6QCcvyA4TLKaF5sbc0jWFoBp5MzOlz9_j6rJ2PSWsIpCi_yLrQO6Qv3vB2Z2HGcti8TRzOlcpYtkAuuLjoqBtxRW1oBoJpfLWrT1B4nbq9_XVXpXNjI9oWrciwKkDSHcseEB7GfSBJSfNb38O7u61t3MWNJVktT7oR92hBnBiXiEGOf8utCd5OgDewj0iQ_u_QOGqtirZ9PWK7cYyill84WrhS38H_d_7hTNJsJWABBnCz1MC0DRak_ZTydfisN2h8afWsDxXYmeQgMVuU1_vsOe99RMLfcbCmctuve4I5yFZjyZR13tuYsc8O3PtqRh4-KEZW46CPgVUSe8RwYuQEIsXRyXdb7tALTmLr9_AgbFrUp3LCoKb7IRbo7b0oP-JbGL4QfMrMg3EPOc7QY5c8U9nTOIfVSA0yAR3aFJxvbMfgB1_8OFPhUkQn-Ijqia4tfW5q8fBKZdRwS4duMeXQNUVTs22aKrEHDCfBGglY-ezVWIp7DEaFgSvWQeN2werSTbXPTR49hU6BLJUklJEVVvLOp_AXUK9qQVTLwZgwrLrE4wm4NgFDIxahaK4-t99R721Mmo_png6mAWThNOk-j1qaxmy-mfAQEnoK3e0HtKCo1pou9Kh5YNAWHk7AfYzqp3Va1azH42-HWw12KZ5izW_AW-qkSluhb3yWPzEr4Hx9o3ezBW7qrnagGrbt2-Fbt2mxY6U0cdZtKp-Dd1KcwqBnwKWE45Y4cbauYmDobQWq1iKM37BYWKQlsmFHUZ4E3ffl6UR2dI5MKxN-tGuezZYEYJW6WWYd32B-Qszm6zcYADpKzOC7UD5e-rRdCPo7k2PGf1","expiry":"2025-02-19T17:11:54.8159589-05:00"}
#website link: https://pennstateoffice365.sharepoint.com/sites/AISSeniorDesignProject

2025/02/19 21:34:08 NOTICE: Config file "/root/.config/rclone/rclone.conf" not found - using defaults
No remotes found, make a new one?
n) New remote
s) Set configuration password
q) Quit config
n/s/q> n

Enter name for new remote.
name> onedrive

Option Storage.
Type of storage to configure.
Choose a number from below, or type in your own value.
 1 / 1Fichier
   \ (fichier)
 2 / Akamai NetStorage
   \ (netstorage)
 3 / Alias for an existing remote
   \ (alias)
 4 / Amazon S3 Compliant Storage Providers including AWS, Alibaba, ArvanCloud, Ceph, ChinaMobile, Cloudflare, DigitalOcean, Dreamhost, GCS, HuaweiOBS, IBMCOS, IDrive, IONOS, LyveCloud, Leviia, Liara, Linode, Magalu, Minio, Netease, Outscale, Petabox, RackCorp, Rclone, Scaleway, SeaweedFS, Selectel, StackPath, Storj, Synology, TencentCOS, Wasabi, Qiniu and others
   \ (s3)
 5 / Backblaze B2
   \ (b2)
 6 / Better checksums for other remotes
   \ (hasher)
 7 / Box
   \ (box)
 8 / Cache a remote
   \ (cache)
 9 / Citrix Sharefile
  

In [ ]:
!sudo mkdir /content/onedrive
# Create onedrive directory in colab

In [ ]:
# Install fusermount3 to ensure proper mounting
!sudo apt-get -y install fuse3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following package was automatically installed and is no longer required:
  libfuse2
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libfuse3-3
The following packages will be REMOVED:
  fuse
The following NEW packages will be installed:
  fuse3 libfuse3-3
0 upgraded, 2 newly installed, 1 to remove and 20 not upgraded.
Need to get 106 kB of archives.
After this operation, 264 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfuse3-3 amd64 3.10.5-1build1 [81.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fuse3 amd64 3.10.5-1build1 [24.7 kB]
Fetched 106 kB in 2s (66.7 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line

In [ ]:
!nohup rclone --vfs-cache-mode full mount onedrive: /content/onedrive &
# After these commands, we should have a onedrive folder in the files section
# within colab with everything we need!

nohup: appending output to 'nohup.out'


## Dataset Setup:

Now that the dataset is mount we need to copy the files from the datasets taht we want to use for creating our model

Note: This will consume a lot of disk resources on Colab.

In [ ]:
# Import utilities to take random image sampling and copy to target directory
import os
import shutil
import random

# Create a new directory specifically to hold dataset
!sudo mkdir /content/SenDsgnTraining

In [ ]:
# Code assistance from Joey George
# Dataset path:
source_dir = r"/content/onedrive/Training Images/Training"
target_dir = r"/content/SenDsgnTraining"

# Define classes for the smaller datasets
# The options we have are:
# "narrowleaf_cattail", "none", "phragmites", "purple_loosestrife"
classes = ["none", "purple_loosestrife", "phragmites", "narrowleaf_cattail"]
splits = {"train": 70, "val": 15, "test": 15}  # Percentage split

In [ ]:
# Create target directory structure
for split in splits:
    for class_name in classes:
        os.makedirs(os.path.join(target_dir, split, class_name), exist_ok=True)

In [ ]:
# Code addistance from Landon Grafton
# Process each class
for class_name in classes:
    source_path = os.path.join(source_dir, class_name)
    images = [f for f in os.listdir(source_path) if f.endswith(".jpg")]

    # Randomly sample 100 images
    selected_images = random.sample(images, min(100, len(images)))

    # Distribute images across train, val, test
    split_counts = {k: int(v * len(selected_images) / 100) for k, v in splits.items()}
    start_idx = 0
    for split, count in split_counts.items():
        split_path = os.path.join(target_dir, split, class_name)
        for i in range(count):
            img_name = f"{start_idx}_{class_name}.jpg"
            shutil.copy(os.path.join(source_path, selected_images[start_idx]), os.path.join(split_path, img_name))
            start_idx += 1

##Training

With the dataset properly taken care of through sharepoint method, the training can begin.

The best thing about the classification model is that no .yaml file is necessary, as the folder structure is enough to ensure the training, validation, and testing splits are recognized for each class.

In [ ]:
# Install Ultralytics and rdependencies
%pip install ultralytics  # install
from ultralytics import YOLO, checks
checks()  # checks

Ultralytics 8.3.77 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.0/235.7 GB disk)


In [ ]:
# Load the pretrained Yolov8 Model
model = YOLO("yolov8s-cls.pt")

100%|██████████| 12.3M/12.3M [00:00<00:00, 17.2MB/s]


In [ ]:
# Train the model
results = model.train(data=target_dir, epochs=10, imgsz=640)

Ultralytics 8.3.77 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=classify, mode=train, model=yolov8s-cls.pt, data=/content/SenDsgnTraining, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

train: Scanning /content/SenDsgnTraining/train... 140 images, 0 corrupt: 100%|██████████| 140/140 [00:00<?, ?it/s]
val: Scanning /content/SenDsgnTraining/val... 30 images, 0 corrupt: 100%|██████████| 30/30 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/classify/train2
Starting training for 10 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00,  9.10it/s]

                   all          1          1


...

      10/10      2.54G    0.02544         12        640: 100%|██████████| 9/9 [00:48<00:00,  5.38s/it]
               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00, 10.49it/s]

                   all          1          1



10 epochs completed in 0.156 hours.
Optimizer stripped from runs/classify/train2/weights/last.pt, 10.3MB
Optimizer stripped from runs/classify/train2/weights/best.pt, 10.3MB

Validating runs/classify/train2/weights/best.pt...
Ultralytics 8.3.77 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,077,762 parameters, 0 gradients, 12.4 GFLOPs
train: /content/SenDsgnTraining/train... found 140 images in 2 classes ✅ 
val: /content/SenDsgnTraining/val... found 30 images in 2 classes ✅ 
test: /content/SenDsgnTraining/test... found 30 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00,  7.04it/s]


                   all          1          1
Speed: 0.8ms preprocess, 2.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2


In [ ]:
# For further uses where we want to download the "runs" folder and save the training results
!zip -r /content/run1.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/classify/ (stored 0%)
  adding: content/runs/classify/train2/ (stored 0%)
  adding: content/runs/classify/train2/confusion_matrix.png (deflated 36%)
  adding: content/runs/classify/train2/train_batch2.jpg (deflated 1%)
  adding: content/runs/classify/train2/val_batch0_labels.jpg (deflated 0%)
  adding: content/runs/classify/train2/args.yaml (deflated 52%)
  adding: content/runs/classify/train2/results.png (deflated 12%)
  adding: content/runs/classify/train2/results.csv (deflated 64%)
  adding: content/runs/classify/train2/train_batch1.jpg (deflated 1%)
  adding: content/runs/classify/train2/confusion_matrix_normalized.png (deflated 34%)
  adding: content/runs/classify/train2/weights/ (stored 0%)
  adding: content/runs/classify/train2/weights/best.pt (deflated 8%)
  adding: content/runs/classify/train2/weights/last.pt (deflated 8%)
  adding: content/runs/classify/train2/val_batch0_pred.jpg (deflated 0%)
  adding: content/runs/c